<a href="https://colab.research.google.com/github/mashdas/nlp_101/blob/master/nsl_preprocessed_graph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
##WE DON'T HAVE A GRAPH IN THIS SCENARIO
##HERE IP vectors are plotted into the embedding space.The one near to one another 
##are considered as neighbors and a graph is constructed 

!pip install neural_structured_learning
import neural_structured_learning as nsl
import tensorflow as tf
import tensorflow_hub as hub##Check


In [0]:
imdb=tf.keras.datasets.imdb
pp_train_data,pp_train_labels=(imdb.load_data(num_words=10000))

In [0]:
##Embedding-->to find which review is similar to another,i.e finding neighbors

In [0]:
pretrained_embedding="https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"

In [0]:
hub_layer=hub.KerasLayer(pretrained_embedding,input_shape=[],dtype=tf.string,trainable=True)


In [0]:


##Generating the embedding...think of it as projecting the imdb 
#reviews into a "embedding" space of sorts..the ones nearest are the neighbors
record_id=int(0)
with tf.io.TFRecordWriter('/tmp/imdb/embeddings.tfr') as writer:
  for word_vector in pp_train_data:
    text=decode_review(word_vector)
    sentence_embedding=hub_layer(tf.reshape(text,shape=[-1]))
    sentence_embedding=tf.reshape(sentence_embedding,shape=[-1])
    write_embedding_example(sentence_embedding,record_id)
    record_id+=1
##verify decode_review function purpose with Mr.M   

In [0]:
##Build graph function from embedding

nsl.tools.build_graph(['/tmp/imdb/embeddings.tfr'],'/tmp/imdb/graph_99.tsv',similarity_threshold=0.8)
##graph_99.tsv is missing..also what does it do ?

##THIS PART IS NOT CLEAR TO ME
##CREATTING Examle Features

next_record_id=create_example(pp_train_data,pp_train_labels,'/tmp/imdb/train_data.tfr',starting_record_id=0)
create_example(pp_test_data,pp_test_labels,'/tmp/imdb/test_data.tfr',starting_record_id=next_record_id)
##HAVE NO IDEA WHAT's GOING ON WITH THE ABOVE TWO LINES
##WILL ASK Mr.MORONEY WHEN I GET THE CHANCE


In [0]:
##Augmentimg training data by merging neighbors into sample features

nsl.tools.pack_nbrs('/tmp/imdb/train_data.tfr',
                    '/tmp/imdb/graph_99.tsv',
                    '/tmp/imdb/nsl_train_data.tfr',
                    add_unidirected_edges=True,max_nbrs=3)

In [0]:
##Extract features

train_ds,test_ds=make_dataset('/tmp/imdb/nsl_train.tfr','/tmp/imdb/test.tfr')

In [0]:
##Base model..could be any model
from tensorflow.keras import Sequential

In [0]:
base_model=Sequential()
base_model.add()##YOUR CHOICE
##BUILD ONE !


In [0]:
##WRAP the base model with graph regularization

graph_config=nsl.configs.GraphReConfig(neighbor_config=nsl.configs.GraphNeighborConfig(max_neighbors=2))
graph_model=nsl.keras.GraphRegularization(base_model,graph_config)

In [0]:
##COMPILE,TRAIN,EVALUATE
graph_model.compile(opimizer="adam",loss="sparse_categorical_crossentorpy",metrics=["accuracy"])
graph_model.fit(train_ds,epochs=2)
graph_model.evaluate(test_ds)